In [1]:
import random
import numpy as np
import glob
import os
from pydicom import dcmread
import shutil

In [2]:
source_base_dir = '/Users/E119562/Downloads/Test-GBM'
destination_base_dir = '/Users/E119562/Downloads/Test-GBM-Anon'

In [3]:
patient_dirs_l = [ name for name in os.listdir(source_base_dir) if os.path.isdir(os.path.join(source_base_dir, name)) ]
patient_dirs_l

['190854_KLEIN^ANTHONY^TREVOR^MR', '210523_GALLOWAY^Samuel']

In [4]:
base_patient_id = 1

In [32]:
mapping_l = []
for patient_idx,patient_dir in enumerate(patient_dirs_l):
    patient_id = int(patient_dir.split('_')[0])
    anon_patient_id = patient_idx+base_patient_id
    anon_patient_folder_name = 'Brain-{:04d}'.format(anon_patient_id)
    mapping_l.append({'source_base_dir':source_base_dir, 'patient_dir':patient_dir, 'patient_id':patient_id, 'destination_base_dir':destination_base_dir, 'anon_patient_dir':anon_patient_folder_name, 'anon_patient_index':anon_patient_id})
    patient_full_dir = source_base_dir + os.sep + patient_dir
    print(patient_full_dir)
    dicom_files = glob.glob('{}/**/*.dcm'.format(patient_full_dir), recursive=True)
    print('found {} source files'.format(len(dicom_files)))
    for source_file in dicom_files:
        rel_path = os.path.relpath(source_file, patient_full_dir)
        target_file = destination_base_dir + os.sep + anon_patient_folder_name + os.sep + rel_path
        # load and process the file
        try:
            ds = dcmread(source_file)
        except Exception as e:
            print(e)
            invalid_file_count += 1
        else:
            # create the anon folder if it doesn't exist
            target_dir = os.path.dirname(target_file)
            if not os.path.exists(target_dir):
                os.makedirs(target_dir)
            ds.save_as(target_file)


/Users/E119562/Downloads/Test-GBM/190854_KLEIN^ANTHONY^TREVOR^MR
found 7063 source files
/Users/E119562/Downloads/Test-GBM/210523_GALLOWAY^Samuel
found 756 source files


In [33]:
import pandas as pd

In [34]:
mapping_l

[{'source_base_dir': '/Users/E119562/Downloads/Test-GBM',
  'patient_dir': '190854_KLEIN^ANTHONY^TREVOR^MR',
  'patient_id': 190854,
  'destination_base_dir': '/Users/E119562/Downloads/Test-GBM-Anon',
  'anon_patient_dir': 'Brain-0001',
  'anon_patient_index': 1},
 {'source_base_dir': '/Users/E119562/Downloads/Test-GBM',
  'patient_dir': '210523_GALLOWAY^Samuel',
  'patient_id': 210523,
  'destination_base_dir': '/Users/E119562/Downloads/Test-GBM-Anon',
  'anon_patient_dir': 'Brain-0002',
  'anon_patient_index': 2}]

In [35]:
df = pd.DataFrame(mapping_l)

In [36]:
df

,source_base_dir,patient_dir,patient_id,destination_base_dir,anon_patient_dir,anon_patient_index
0,/Users/E119562/Downloads/Test-GBM,190854_KLEIN^ANTHONY^TREVOR^MR,190854,/Users/E119562/Downloads/Test-GBM-Anon,Brain-0001,1
1,/Users/E119562/Downloads/Test-GBM,210523_GALLOWAY^Samuel,210523,/Users/E119562/Downloads/Test-GBM-Anon,Brain-0002,2


In [37]:
mapping_file = source_base_dir + os.sep + 'dicom-anon-mapping.xlsx'
df.to_excel(mapping_file)

In [38]:
dff = pd.read_excel(mapping_file, index_col=0)

In [39]:
dff

,source_base_dir,patient_dir,patient_id,destination_base_dir,anon_patient_dir,anon_patient_index
0,/Users/E119562/Downloads/Test-GBM,190854_KLEIN^ANTHONY^TREVOR^MR,190854,/Users/E119562/Downloads/Test-GBM-Anon,Brain-0001,1
1,/Users/E119562/Downloads/Test-GBM,210523_GALLOWAY^Samuel,210523,/Users/E119562/Downloads/Test-GBM-Anon,Brain-0002,2


In [40]:
dff['anon_patient_index'].max()

2